In [1]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 2.6MB 4.1MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 1.4MB 4.2MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 931kB 4.0MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
     |████████████████████████████████| 389kB 4.1MB/s 
     |████████████████████████████████| 225kB 4.4MB/s 
     |████████████████████████████████| 235kB 7.1MB/s 
     |████████████████████████████████| 51kB 4.2MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.7.2-cp37-none-any.whl size=388143 sha256=ba5d89407be2e2d58a3d1bc5fe2c00cbe523bebcf56c197253c1ec5332023961
  Stored in directory: /root/.cache/pip/wheels/e7/66/5b/ad17ef7f04b7c425dc6930daac160c3747231b0d65f9ac7972
Successfully built torch-geometric


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
from torch_geometric.utils import to_dense_adj

import torch_geometric.transforms as T


from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import tqdm

In [3]:
import torch_geometric.datasets
device = torch.device('cuda:0')
print(device)
data = torch_geometric.datasets.amazon.Amazon('.','Computers').data.to(device)
# data = torch_geometric.datasets.amazon.Amazon('.','Computers').data.to(device)

cuda:0
Processing...
Done!


In [4]:
class GNNEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim,output_dim):
        super(GNNEncoder, self).__init__()

        self.conv1 = pyg_nn.GCNConv(input_dim, hidden_dim*4)
        self.conv2 = pyg_nn.GCNConv(hidden_dim*4, hidden_dim*2)
        self.conv3 = pyg_nn.GCNConv(hidden_dim*2, hidden_dim)

        self.ln1 = nn.Linear(hidden_dim*4,hidden_dim*4)
        self.ln2 = nn.Linear(hidden_dim*2,hidden_dim*2)
        self.ln3 = nn.Linear(hidden_dim,output_dim)

        self. bias = nn.Parameter(torch.tensor(-10,dtype = float))

        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(1)
        self.dropout = 0.25



    def forward(self, data, do_mult = True):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x,edge_index)
        x = F.relu(x)
        x = F.tanh(self.ln1(x))+x

        x = self.conv2(x,edge_index)
        x = F.relu(x)
        x = F.tanh(self.ln2(x))+x

        x = self.conv3(x,edge_index)
        x = F.relu(x)
        x = self.ln3(x)

        if do_mult:
            x = (x@torch.transpose(x, 0, 1))
            shape = x.shape
            x = self.bn2(x.view(-1,1)).view(shape)
            x = torch.sigmoid(x)

        return x



class GNNDecoder(nn.Module):
    def __init__(self, input_dim, hidden_dim,ouput_dim):
        super(GNNDecoder, self).__init__()

        self.ln11 = nn.Linear(input_dim,hidden_dim*2)
        self.ln12 = nn.Linear(input_dim,hidden_dim*2)

        self.ln2 = nn.Linear(hidden_dim*4,hidden_dim)
        self.ln3 = nn.Linear(hidden_dim,ouput_dim)

        self.bn1 = nn.BatchNorm1d(input_dim)
        self.bn2 = nn.BatchNorm1d(input_dim)



    def forward(self, x1, x2):
        # x1 = self.bn1(x1)
        # x2 = self.bn2(x2)

        x1 = self.ln11(x1)
        x1 = F.relu(x1)

        x2 = self.ln12(x2)
        x2 = F.relu(x2)

        x = torch.cat([x1,x2],1)

        x = self.ln2(x)
        x = F.relu(x)
        x = self.ln3(x)
        x = torch.sigmoid(x)

        return x


def loss(pred,ture):
    return -10*((2*ture*torch.log(pred+10e-6)).mean() + ((1-ture)*(torch.log(1-pred+10e-6))).mean())


In [5]:
torch.cuda.empty_cache()

A = to_dense_adj(data.edge_index).to(device)
A = A.view(A.shape[1:])

In [6]:
enc = GNNEncoder(data.x.shape[1],32,32*4).to(device)

# loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10]).to(device))

In [29]:
dec = GNNDecoder(32*4,16,1).to(device)

In [8]:
opt = optim.Adam(enc.parameters(), lr=0.005)
enc.train(True)

for ep in range(8000):
    pred = enc(data)

    pre = (A*pred).sum()/pred.sum()
    re  = (A*pred).sum()/A.sum()
    l = loss(pred.view(-1),A.view(-1))
        
    opt.zero_grad()
    l.backward()
    opt.step()

    if ep%10 == 0:
        print(ep,f' Loss: {float(l):.6f}  >>  {float(pre):.6f}  {float(re):.6f} ')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0  Loss: 7.841780  >>  0.004019  0.748952 
10  Loss: 7.369096  >>  0.004130  0.751122 
20  Loss: 7.061755  >>  0.004132  0.735101 
30  Loss: 6.552155  >>  0.003946  0.685026 
40  Loss: 6.141775  >>  0.003666  0.623264 
50  Loss: 5.881571  >>  0.003513  0.583384 
60  Loss: 5.621598  >>  0.003209  0.521065 
70  Loss: 5.393567  >>  0.002832  0.449489 
80  Loss: 5.197146  >>  0.002782  0.430751 
90  Loss: 5.013518  >>  0.003010  0.453179 
100  Loss: 4.839059  >>  0.003027  0.443550 
110  Loss: 4.671912  >>  0.003014  0.429676 
120  Loss: 4.512549  >>  0.003058  0.423881 
130  Loss: 4.360133  >>  0.003070  0.413835 
140  Loss: 4.214393  >>  0.003117  0.408645 
150  Loss: 4.074939  >>  0.003129  0.399046 
160  Loss: 3.941491  >>  0.003151  0.390792 
170  Loss: 3.813767  >>  0.003171  0.382601 
180  Loss: 3.691556  >>  0.003193  0.374817 
190  Loss: 3.574578  >>  0.003219  0.367623 
200  Loss: 3.462591  >>  0.003245  0.360549 
210  Loss: 3.355352  >>  0.003274  0.354037 
220  Loss: 3.252664  

KeyboardInterrupt: ignored

In [33]:
opt = optim.Adam(dec.parameters(), lr=0.00005)
loss = nn.BCELoss()
batch_size = 256*16
enc.train(False)
emb = enc(data,False)
emb = emb.detach()

emb = emb/emb.var() - emb.mean()

for ep in range(1000):
    

    indices = torch.randint(0,data.edge_index.shape[1],(int(batch_size/2),))
    x1 = emb[data.edge_index[0,indices]]
    x2 = emb[data.edge_index[1,indices]]
    


    num_neg_smp = 0
    tx1 = []
    tx2 = []
    for i,j in torch.randint(0,emb.shape[0],(batch_size*4,2)):
        if A[i,j] == 0 and num_neg_smp < batch_size/2:
            tx1.append(emb[i])
            tx2.append(emb[j])
            num_neg_smp += 1
        if num_neg_smp > batch_size/2:
            break

    perm = torch.randperm(batch_size)
    x1 = torch.cat([x1,torch.stack(tx1).to(device)],0)
    x1 = x1[perm]
    x2 = torch.cat([x2,torch.stack(tx2).to(device)],0)
    x2 = x2[perm]
    
    y = torch.cat([torch.ones([int(batch_size/2)]),torch.zeros([int(batch_size/2)])],0).to(device)
    y = y[perm]
 

    pred1 = dec(x1,x2).view(-1)
    pred2 = dec(x2,x1).view(-1)
    # print(pred1.min(),pred1.max())
    opt.zero_grad()
    # pre = (y*pred1).sum()/(pred1.sum()+10e-6)
    # re  = (y*pred1).sum()/(y.sum()+10e-6)
    # l = -(re*pre)/(pre+ re)
    l = loss(pred1,y) + loss(pred2,y)

    l.backward()
        
    opt.step()

    if ep%10 == 0:
        pre = (y*pred1).sum()/(pred1.sum()+10e-6)
        re  = (y*pred1).sum()/(y.sum()+10e-6)
        print(ep,f' Loss: {float(l):.6f}  >>  {float(pre):.6f}  {float(re):.6f} ')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0  Loss: 0.587055  >>  0.812423  0.806773 
10  Loss: 0.623688  >>  0.800251  0.794997 
20  Loss: 0.625682  >>  0.804427  0.798473 
30  Loss: 0.584389  >>  0.815027  0.803516 
40  Loss: 0.656750  >>  0.797663  0.795675 
50  Loss: 0.587456  >>  0.808156  0.813364 
60  Loss: 0.638162  >>  0.796838  0.791502 
70  Loss: 0.648301  >>  0.798887  0.793970 
80  Loss: 0.641511  >>  0.798966  0.792416 
90  Loss: 0.600198  >>  0.800879  0.808404 
100  Loss: 0.627120  >>  0.801716  0.795159 
110  Loss: 0.621997  >>  0.800191  0.802906 
120  Loss: 0.630683  >>  0.799267  0.797689 
130  Loss: 0.596155  >>  0.803791  0.808712 
140  Loss: 0.630822  >>  0.797475  0.797440 
150  Loss: 0.603256  >>  0.807694  0.798866 
160  Loss: 0.622459  >>  0.796801  0.802240 
170  Loss: 0.617845  >>  0.802185  0.803489 
180  Loss: 0.661157  >>  0.791817  0.801703 
190  Loss: 0.610825  >>  0.804092  0.796150 
200  Loss: 0.600653  >>  0.807569  0.795699 
210  Loss: 0.629090  >>  0.795383  0.806847 
220  Loss: 0.564348  

In [28]:
y.mean()

tensor(0.5000, device='cuda:0')

In [ ]:
import tqdm 
A.cpu()
AP = torch.empty(A.shape)
for i in tqdm.trange(len(data.x)):      
    A[i,:] = dec(torch.stack([emb[i]]*len(data.x)),emb).view(-1)
        



  0%|          | 0/3327 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
AP*(1-A).sum()

torch.Size([512, 128])

In [ ]:
        pred = enc(data).view(-1) >0.9
pre = (A*pred).sum()/pred.sum()
re  = (A*pred).sum()/A.sum()
pre, re

In [ ]:
f'Loss: {float(1):.2f} >>  {float(1):.2f} {float(1):.2f}'

In [ ]:
pred = torch.where(model(data)>0.3,1,0)

pre = (A*pred).sum()/pred.sum()
re  = (A*pred).sum()/A.sum()
# pre* re/(pre + re)
pre,re

NameError: ignored

In [ ]:
torch.randperm(9)[:-(9%2)].view(-1,2)

tensor([[1, 7],
        [3, 5],
        [2, 8],
        [4, 6]])